## Palabras
modelo.cluster_word_distribution
## Documentos
<ol><li> Por frecuencia:
<ol><li> Extraer palabras clave del clúster</li>
<li> Contar coincidencias y calcular puntuación de relevancia:puntuacion += palabras_clave_ponderadas[str(palabra)]  </li>
<li>Ordenar documentos por puntuación de relevancia</li></ol></li>
<li> Por Tf-idf   
<ol><li>Extraer palabras clave del clúster</li>
     <li> Obtener índices de palabras clave en la matriz TF-IDF
    <ol><li>TF-IDF medio en el corpus:palabras_clave_ponderadas[palabra] = np.mean(tfidf_matrix[:, idx].toarray()) </li> 
    <li>Calcular la media solo dentro del clúster: palabras_clave_ponderadas[palabra] = np.mean(tfidf_matrix[indices_cluster, idx].toarray()) </li></ol></li>
    <li> Por Tf-idf por cluster en vez de por documento <strong>(opción final elegida)</strong></li>  

</ol>

In [2]:
import pandas as pd
import pickle
from IPython.display import display, HTML
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.corpora import Dictionary, MmCorpus

In [3]:
with open('recurso/resultados_modelos.pkl', 'rb') as archivo:
    resultados = pickle.load(archivo)
    
# Para bigramas    
# with open('recurso/resultados_modelos_bi.pkl', 'rb') as archivo:
#     resultados = pickle.load(archivo)

In [4]:
resultados["gsdmm_100"]['resultados_ejecuciones']

[{'adjusted_rand_score': 0.20750005048726017,
  'normalized_mutual_info_score': 0.4500232582107795,
  'confusion_matrix': array([[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 9,  0, 49, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  1, ...,  0,  0,  0]], dtype=int64),
  'fowlkes_mallows_score': 0.27961404729597233,
  'bcubed_precision': 0.4643622245288614,
  'bcubed_recall': 0.2202949308471145,
  'bcubed_f1': 0.24446687554129068},
 {'adjusted_rand_score': 0.2026919322006004,
  'normalized_mutual_info_score': 0.44244498826099543,
  'confusion_matrix': array([[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [10,  6,  5, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  2,  3, ...,  0,  0,  0]], dtype=int64),
  'fowlkes_mallows_score': 0.2768484321419279,
  'bcubed

In [4]:
lista_resultados = resultados["gsdmm_100"]['resultados_ejecuciones']

# Crear un DataFrame de pandas
df = pd.DataFrame(lista_resultados)

# Eliminar la columna 'confusion_matrix' ya que no se puede representar directamente en HTML
df = df.drop(columns=['confusion_matrix'])

# Convertir el DataFrame a HTML
tabla_html = df.to_html(index=False)

# Imprimir la tabla HTML
display(HTML(tabla_html))

adjusted_rand_score,normalized_mutual_info_score,fowlkes_mallows_score,bcubed_precision,bcubed_recall,bcubed_f1
0.207500,0.450023,0.279614,0.464362,0.220295,0.244467
0.202692,0.442445,0.276848,0.458252,0.225538,0.237165
0.215328,0.448040,0.287111,0.466228,0.247657,0.253089
0.216156,0.451759,0.289376,0.470597,0.241318,0.254636
0.195635,0.444791,0.268564,0.461074,0.216948,0.234565


In [5]:
# Calcular la media para cada fila
medias = df[['adjusted_rand_score', 'normalized_mutual_info_score', 'bcubed_f1']].mean(axis=1)

# Añadir las medias como una nueva columna al DataFrame
df['Media'] = medias

df_ordenado = df.sort_values(by='Media',ascending=False)

# Crear un nuevo dataframe solo con las medias
df_medias = df_ordenado[['Media']]

# Convertir el DataFrame a HTML
tabla_html_medias = df_medias.to_html()

display(HTML(tabla_html_medias))

,Media
3,0.307517
2,0.305485
0,0.300663
1,0.294101
4,0.291664


In [5]:
etiquetas_predichas=resultados["gsdmm_100"]['etiquetas_predichas'][3]
# Para bigramas
# etiquetas_predichas=resultados["gsdmm"]['etiquetas_predichas'][1]

In [6]:
with open('resultado/modelos_topicos/gsdmm_100_ejecucion_890.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)
# with open('modelos_topicos_bi/gsdmm_ejecucion_123.pkl', 'rb') as archivo:
#     modelo = pickle.load(archivo)

### Obtener las palabras relevantes de los tópicos 

In [33]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts = sorted(
            modelo.cluster_word_distribution[cluster].items(),
            key=lambda k: k[1],
            reverse=True,
        )[:values]
        print('Cluster %s : %s'%(cluster,sort_dicts))
        print(' — — — — — — — — — ')

doc_count = np.array(modelo.cluster_doc_count)
print('Number of documents per topic :', doc_count)
print('*'*20)

# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-21:][::-1]
print('Most important clusters (by number of docs inside):', top_index)
print('*'*20)


# Show the top 10 words in term frequency for each cluster 
top_words(modelo.cluster_word_distribution, top_index, 15)

Number of documents per topic : [  29 1915 1621 1932 2289 1928 1062 1104   41  445   40  345   48 4404
 1268   48 1696   11  178 2358  246 1754 1069  691 2778   21 1239   54
 1009 3976 1864 1398  395  409 2425  100 3098  286  859  668  557 1063
  561   75  935  454 1673   23 1817   95 2064  227   61 1297 1402  121
  388 2184   34  445 3878]
********************
Most important clusters (by number of docs inside): [13 29 60 36 24 34 19  4 57 50  3  5  1 30 48 21 16 46  2 54 31]
********************
Cluster 13 : [('semaforo', 6588), ('bache', 5735), ('calle', 5333), ('solicitar', 4018), ('reparacion', 3845), ('avenida', 3711), ('fundido', 3495), ('paseo', 1846), ('rama', 1532), ('tapar', 1023), ('san', 710), ('plaza', 525), ('poda', 512), ('situado', 509), ('podar', 508)]
 — — — — — — — — — 
Cluster 29 : [('farola', 5030), ('luz', 5026), ('punto', 3509), ('calle', 3410), ('matricular', 3168), ('apagado', 2064), ('alumbrado', 1146), ('fundido', 1061), ('zona', 861), ('saludo', 785), ('ilum

In [6]:
def generar_palabras_topicos_cluster_excel(modelo, lista_topicos, num_words=15, output_excel_path="palabras_topicos_GSDMM.xlsx"):
   

    data = []

    for topic_id in lista_topicos:
        # Obtener las palabras más probables para el tópico actual
        palabras_probables = sorted(
            modelo.cluster_word_distribution[topic_id].items(),
            key=lambda k: k[1],
            reverse=True,
        )[:num_words]

        # Extraer solo las palabras y unirlas en una cadena separada por comas
        palabras = [palabra for palabra, probabilidad in palabras_probables]
        palabras_str = ", ".join(palabras)

        # Agregar la cadena a la lista de datos
        data.append({"Tópico": topic_id, "Palabras": palabras_str})

    # Crear un DataFrame de pandas a partir de los datos
    df = pd.DataFrame(data)

    # Exportar el DataFrame a un archivo Excel
    df.to_excel(output_excel_path, index=False)

    print(f"Las palabras de los tópicos se han exportado a '{output_excel_path}'.")

In [7]:
generar_palabras_topicos_cluster_excel(modelo, top_index)

Las palabras de los tópicos se han exportado a 'palabras_topicos_GSDMM.xlsx'.


## Obtener documentos más representativos

In [9]:
def imprimir_documentos_relevantes_cluster_etiquetas(etiquetas_predichas, documentos_preprocesados,documentos_originales, cluster_id, num_documentos=5, num_palabras_clave=10, mgp=None):


    # 1. Extraer palabras clave del clúster
    palabras_clave = sorted(
        mgp.cluster_word_distribution[cluster_id].items(),
        key=lambda k: k[1],
        reverse=True,
    )[:num_palabras_clave]
    # Ponderación de palabras clave (usando la frecuencia como ponderación)
    palabras_clave_ponderadas = {str(palabra): frecuencia for palabra, frecuencia in palabras_clave}

    # 2. Contar coincidencias y calcular puntuación de relevancia
    puntuaciones_relevancia = []
    for i, documento in enumerate(documentos_preprocesados):
        if etiquetas_predichas[i] == cluster_id:
            puntuacion = 0
            for palabra in documento:
             
                if palabra in palabras_clave_ponderadas:
                    
                    puntuacion += palabras_clave_ponderadas[str(palabra)]
            puntuaciones_relevancia.append((documentos_originales[i], puntuacion))

    # 3. Ordenar documentos por puntuación de relevancia
    puntuaciones_relevancia_ordenadas = sorted(puntuaciones_relevancia, key=lambda x: x[1], reverse=True)[:num_documentos]

    # Imprimir resultados
    print(f"Documentos más relevantes del clúster {cluster_id}:")
    for documento, puntuacion in puntuaciones_relevancia_ordenadas:
        print(f"  Documento: {documento}, Puntuación: {puntuacion}")
        print("-" * 20)

In [8]:
#df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=';')

In [9]:
# documentos=df_preprocesado['texto_preprocesado'].apply(ast.literal_eval) 
# documentos_originales=df_preprocesado['texto']
with open('recursos/documents.pkl', 'rb') as f:
    documentos = pickle.load(f)
with bz2.BZ2File('recursos/documentos_originales.pkl.bz2', 'rb') as f:
    documentos_originales = pickle.load(f)

In [12]:
cluster_a_imprimir = 13  # Imprimir documentos del clúster 0
numero_documentos_a_imprimir = 5 # Imprimir los n documentos más relevantes
imprimir_documentos_relevantes_cluster_etiquetas(etiquetas_predichas, documentos,documentos_originales,cluster_a_imprimir, numero_documentos_a_imprimir, mgp=modelo)

Documentos más relevantes del clúster 13:
  Documento: Baches en la avenida San José con la avenida Cesáreo Alierta Hay varios baches en la avenida San José con la avenida Cesáreo Alierta.
Solicito la reparación de los baches.

La reparación ya se solicitó hace 77 días el 23/11/2022.
La reparación ya se solicitó hace 57 días el 14/12/2022.
La reparación ya se solicitó hace 10 días el 30/01/2023., Puntuación: 63501
--------------------
  Documento: Semáforo fundido en la avenida Madrid entre la calle Don Pedro de Luna y la calle Sangenis En la avenida Madrid entre la calle Don Pedro de Luna y la calle Sangenis el semáforo está fundido.
Solicito la reparación del semáforo., Puntuación: 63371
--------------------
  Documento: Baches en el paseo de Cuéllar con el Centro Regional de Mando Hay varios baches en el paseo de Cuéllar con el Centro Regional de Mando.
Solicito la reparación de los baches.

La reparación ya se solicitó hace 379 días el 22/01/2021.
La reparación ya se solicitó hace 

In [10]:
def filter_documents_for_dictionary(tokenized_documents, filtered_words):
    filtered_documents = []
    for doc in tokenized_documents:
        filtered_doc = [word for word in doc if word in filtered_words]
        filtered_documents.append(filtered_doc)
    return filtered_documents
#df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=';')
# dictionary = Dictionary.load('recurso/dictionary_filtrado_spacy.gensim')
# filtered_words = set(dictionary.token2id.keys())
# preprocesado_documents=df_preprocesado['texto_preprocesado'].apply(ast.literal_eval)    
# documents = filter_documents_for_dictionary(preprocesado_documents, filtered_words)
with open('recursos/documents.pkl', 'rb') as archivo:
    documents = pickle.load(archivo)

# En vez de frecuencia utilizar Tf-idf

In [18]:
vectorizer = TfidfVectorizer()
documentos_como_cadenas = [" ".join(doc) for doc in documents]
# Calcular TF-IDF
tfidf_matrix = vectorizer.fit_transform(documentos_como_cadenas)

In [39]:
def imprimir_documentos_relevantes_cluster_etiquetas_tfidf(etiquetas_predichas, documentos_preprocesados, documentos_originales, cluster_id, num_documentos=5, num_palabras_clave=10, mgp=None, tfidf_matrix=None, vectorizer=None):
    # 1. Extraer palabras clave del clúster
    palabras_clave = sorted(
        mgp.cluster_word_distribution[cluster_id].items(),
        key=lambda k: k[1],
        reverse=True
    )[:num_palabras_clave]

    # 2. Obtener índices de palabras clave en la matriz TF-IDF
    palabras_clave_ponderadas = {}
    if vectorizer and tfidf_matrix is not None:
        feature_names = vectorizer.get_feature_names_out() #lista de palabras (vocabulario) que fueron utilizadas
        for palabra, _ in palabras_clave:
            if palabra in vectorizer.vocabulary_:
                idx = vectorizer.vocabulary_[palabra] # verifica si la palabra está en el vocabulario 
                palabras_clave_ponderadas[palabra] = np.mean(tfidf_matrix[:, idx].toarray())  # TF-IDF medio en el corpus

    # 3. Evaluar documentos del clúster
    puntuaciones_relevancia = []
    for i, documento in enumerate(documentos_preprocesados):
        if etiquetas_predichas[i] == cluster_id:
            puntuacion = sum(palabras_clave_ponderadas.get(palabra, 0) for palabra in documento)
            puntuacion /= len(documento) # Para que los documentos largos no tengan ventaja
            puntuaciones_relevancia.append((documentos_originales[i], puntuacion))

    # 4. Ordenar y mostrar documentos más relevantes
    puntuaciones_relevancia_ordenadas = sorted(puntuaciones_relevancia, key=lambda x: x[1], reverse=True)[:num_documentos]

    print(f"Documentos más relevantes del clúster {cluster_id}:")
    for documento, puntuacion in puntuaciones_relevancia_ordenadas:
        print(f"  Documento: {documento}, Puntuación: {puntuacion}")
        print("-" * 20)


In [ ]:
with bz2.BZ2File('recursos/documentos_originales.pkl.bz2', 'rb') as f:
    documentos_originales = pickle.load(f)
cluster_id =13
imprimir_documentos_relevantes_cluster_etiquetas_tfidf(etiquetas_predichas, documents, documentos_originales, cluster_id, mgp=modelo, tfidf_matrix=tfidf_matrix, vectorizer=vectorizer,num_documentos=20)

In [13]:
def imprimir_documentos_relevantes_cluster_etiquetas_tfidf2(etiquetas_predichas, documentos_preprocesados, documentos_originales, cluster_id, num_documentos=5, num_palabras_clave=10, mgp=None, tfidf_matrix=None, vectorizer=None):
    # 1. Extraer palabras clave del clúster
    palabras_clave = sorted(
        mgp.cluster_word_distribution[cluster_id].items(),
        key=lambda k: k[1],
        reverse=True
    )[:num_palabras_clave]

    # 2. Obtener índices de palabras clave en la matriz TF-IDF
    palabras_clave_ponderadas = {}
    indices_cluster = np.where(etiquetas_predichas == cluster_id)[0]
    if vectorizer and tfidf_matrix is not None:
        feature_names = vectorizer.get_feature_names_out() #lista de palabras (vocabulario) que fueron utilizadas
        for palabra, _ in palabras_clave:
            if palabra in vectorizer.vocabulary_:
                idx = vectorizer.vocabulary_[palabra] # verifica si la palabra está en el vocabulario 
                #palabras_clave_ponderadas[palabra] = np.mean(tfidf_matrix[:, idx].toarray())  # TF-IDF medio en el corpus
                palabras_clave_ponderadas[palabra] = np.mean(tfidf_matrix[indices_cluster, idx].toarray()) #Calcular la media solo dentro del clúster

    # 3. Evaluar documentos del clúster
    puntuaciones_relevancia = []
    for i, documento in enumerate(documentos_preprocesados):
        if etiquetas_predichas[i] == cluster_id:
            puntuacion = sum(palabras_clave_ponderadas.get(palabra, 0) for palabra in documento)
            puntuacion /= len(documento)  # Para que los documentos largos no tengan ventaja
            puntuaciones_relevancia.append((documentos_originales[i], puntuacion))

    # 4. Ordenar y mostrar documentos más relevantes
    puntuaciones_relevancia_ordenadas = sorted(puntuaciones_relevancia, key=lambda x: x[1], reverse=True)[:num_documentos]

    print(f"Documentos más relevantes del clúster {cluster_id}:")
    for documento, puntuacion in puntuaciones_relevancia_ordenadas:
        print(f"  Documento: {documento}, Puntuación: {puntuacion}")
        print("-" * 20)


# TFIDF por cluster (opción elegida)

In [76]:
import numpy as np
import pandas as pd
from collections import defaultdict
from numpy.linalg import norm

def imprimir_documentos_relevantes_clusters(etiquetas_predichas, documentos_preprocesados, documentos_originales, top_index, num_documentos=5, nombre_archivo="documentos_relevantes.xlsx"):
      
    resultados_totales = []

    for cluster_id in top_index:
        documentos_cluster = [(i, documentos_preprocesados[i]) for i, etiqueta in enumerate(etiquetas_predichas) if etiqueta == cluster_id]

        # Calcular TF
        tf_cluster = defaultdict(int)
        for _, documento in documentos_cluster:
            for palabra in documento:
                tf_cluster[palabra] += 1

        # Calcular IDF
        palabras_por_cluster = defaultdict(set)
        for i, etiqueta in enumerate(etiquetas_predichas):
            palabras_por_cluster[etiqueta].update(documentos_preprocesados[i])

        num_clusters = len(set(etiquetas_predichas))
        num_clusters_con_palabra = {
            palabra: sum(1 for palabras in palabras_por_cluster.values() if palabra in palabras)
            for palabra in tf_cluster
        }

        idf_cluster = {palabra: np.log((1 + num_clusters) / (1 + num_clusters_con_palabra[palabra])) + 1 for palabra in tf_cluster}
        
        peso_palabras = {palabra: tf_cluster[palabra] * idf_cluster[palabra] for palabra in tf_cluster}

        # Evaluar documentos del clúster
        puntuaciones_relevancia = []
        for doc_id, documento in documentos_cluster:
            # Calcular la frecuencia relativa de cada palabra
            total_palabras_doc = len(documento)
            puntuacion = np.array([peso_palabras.get(palabra, 0) * (documento.count(palabra) / total_palabras_doc) for palabra in documento])

            # Cortar los primeros 300 caracteres del documento original
            documento_recortado = documentos_originales[doc_id][:300]

            puntuaciones_relevancia.append((doc_id, cluster_id, documento_recortado, puntuacion_final))

        # Ordenar por puntuación
        puntuaciones_relevancia_ordenadas = sorted(puntuaciones_relevancia, key=lambda x: x[3], reverse=True)[:num_documentos]

        # Añadir resultados de este cluster
        resultados_totales.extend(puntuaciones_relevancia_ordenadas)

    # Verificar que hay datos antes de crear el DataFrame
    if resultados_totales:
        df = pd.DataFrame(resultados_totales, columns=["ID", "ID de Cluster", "Documento (200 caracteres)", "Puntuación"])

        # Guardar en formato CSV, XLSX o ODS según el nombre de archivo
        if nombre_archivo.endswith('.csv'):
            df.to_csv(nombre_archivo, index=False, encoding="utf-8")
        elif nombre_archivo.endswith('.ods'):
            df.to_excel(nombre_archivo, index=False, engine="odf")
        else:
            df.to_excel(nombre_archivo, index=False, engine="openpyxl")
        print(f"\n Archivo guardado: {nombre_archivo}")
    else:
        print("\n No hay documentos relevantes para los clusters indicados.")


In [ ]:
imprimir_documentos_relevantes_clusters(etiquetas_predichas, documents, documentos_originales, top_index, num_documentos=5)

In [78]:
top_index

array([13, 29, 60, 36, 24, 34, 19,  4, 57, 50,  3,  5,  1, 30, 48, 21, 16,
       46,  2, 54, 31], dtype=int64)